In [0]:
%run "./config"

In [0]:
spark.conf.set(
    "fs.azure.account.key.mystore1984.dfs.core.windows.net",
    f'{key}'
)

In [0]:
dbutils.widgets.text('Date','')
input_value= dbutils.widgets.get('Date')

In [0]:
# loading product category data
df_category=spark.read.csv(f"{raw_folder_path}/product.csv", header= True, inferSchema= True)

In [0]:
from pyspark.sql.functions import lit
df_category=df_category.withColumnRenamed("Name","Category Name")\
                       .drop('rowguid','ModifiedDate')\
                       .withColumn("Date",lit(input_value))

In [0]:
df_category.show(2)

+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-----------+----------------+--------------------+----------------------+----+
|ProductID|       Category Name|ProductNumber|Color|StandardCost|ListPrice|Size| Weight|ProductCategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|      ThumbNailPhoto|ThumbnailPhotoFileName|Date|
+---------+--------------------+-------------+-----+------------+---------+----+-------+-----------------+--------------+-------------------+-----------+----------------+--------------------+----------------------+----+
|      680|HL Road Frame - B...|   FR-R92B-58|Black|     1059.31|   1431.5|  58|1016.04|               18|             6|2002-06-01 00:00:00|       NULL|            NULL|R0lGODlhUAAxAPcAA...|  no_image_availabl...|    |
|      706|HL Road Frame - R...|   FR-R92R-58|  Red|     1059.31|   1431.5|  58|1016.04|               18|             6

In [0]:
# loading product data
df_product =spark.read.csv(f"{raw_folder_path}/product category.csv", header= True, inferSchema= True)

In [0]:
df_prodcat= df_product.join(df_category,df_product.ProductCategoryID== df_category.ProductCategoryID ).drop(df_product.ProductCategoryID)
df_prodcat.columns

['ParentProductCategoryID',
 'Name',
 'rowguid',
 'ModifiedDate',
 'ProductID',
 'Category Name',
 'ProductNumber',
 'Color',
 'StandardCost',
 'ListPrice',
 'Size',
 'Weight',
 'ProductCategoryID',
 'ProductModelID',
 'SellStartDate',
 'SellEndDate',
 'DiscontinuedDate',
 'ThumbNailPhoto',
 'ThumbnailPhotoFileName',
 'Date']

In [0]:
# join
df_product_data= df_prodcat.select('ProductID','Name','StandardCost','ListPrice','Weight','Size','Category Name', 'Date')\
                           .withColumnRenamed("Name","Product Name")
df_product_data.show(2)

+---------+------------+------------+---------+-------+----+--------------------+----+
|ProductID|Product Name|StandardCost|ListPrice| Weight|Size|       Category Name|Date|
+---------+------------+------------+---------+-------+----+--------------------+----+
|      680| Road Frames|     1059.31|   1431.5|1016.04|  58|HL Road Frame - B...|    |
|      706| Road Frames|     1059.31|   1431.5|1016.04|  58|HL Road Frame - R...|    |
+---------+------------+------------+---------+-------+----+--------------------+----+
only showing top 2 rows


In [0]:
product_data_processed= df_product_data.write.mode( "overwrite").parquet(f"{processed_folder_path}/productdata")